In [8]:
from pathlib import Path
import json
from PIL import Image
import os

In [9]:
dataset_dir = Path("..")


# Define the images directory and duplicates directory using Path objects
image_dir = dataset_dir / "images"
meta_dir = dataset_dir / "metadata"
label_dir = dataset_dir / "labels"
yolo_dir = dataset_dir / "yolo"
yolo_label_dir = yolo_dir / "labels"
yolo_image_dir = yolo_dir / "images"
yolo_label_dir.mkdir(exist_ok=True, parents=True)
if not yolo_image_dir.exists():
    os.symlink(image_dir, yolo_dir / "images", target_is_directory=True)

# List all objects in the image directory
objs = sorted([obj.name for obj in image_dir.iterdir()])

with open(meta_dir / "id_to_name.json", "r") as f:
    id_to_name = json.load(f)

with open(meta_dir / "near_duplicates.json", "r") as f:
    near_duplicates = json.load(f)

with open(label_dir / "labels_nms.json", "r") as f:
    labels_nms = json.load(f)

## yolo format

In [10]:
with open(meta_dir / "classes.json", "r") as f:
    class_dict = json.load(f)

for k, v in class_dict.items():
    print(f"  {v}: {k}")

  0: articulated dump truck
  1: bulldozer
  2: combined piling and drilling rig
  3: crawler crane
  4: crawler excavator
  5: crawler loader
  6: duty cycle crane
  7: gantry crane
  8: log loader
  9: maritime crane
  10: material handling machine
  11: mining bulldozer
  12: mining excavator
  13: mining truck
  14: mobile crane
  15: pipelayer
  16: pontoon excavator
  17: reachstacker
  18: telescopic handler
  19: tower crane
  20: truck mixer
  21: wheel excavator
  22: wheel loader


In [11]:
image_list = []
for id, ann in labels_nms.items():
    if ann["boxes"] == []:  # Skip images with no annotations
        continue
    obj = id_to_name[id + ".jpg"].split("/")[0]
    image_list.append(f"./images/{obj}/{id}.jpg")
    boxes_xywh = ann["boxes"]
    classes = ann["classes"]
    cls_ids = [class_dict[class_name] for class_name in classes]

    txt = ""
    # yolo format: "class_id x y w h"
    for i in range(len(boxes_xywh)):
        cls_id = cls_ids[i]
        box = boxes_xywh[i]
        txt += f"{cls_id} {box[0]:.6f} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f}\n"

    # save
    (yolo_label_dir / obj).mkdir(exist_ok=True, parents=True)
    with open(yolo_label_dir / obj / (id + ".txt"), "w") as f:
        f.write(txt)

print(len(image_list))

15270
